In [1]:
from pipeline import DetectBERTPythonPipeline, read_file
from utils import *
import os



/home/jupyter-iec_chau/miniconda3/envs/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jupyter-iec_chau/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/jupyter-iec_chau/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/jupyter-iec_chau/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `to

In [2]:
from DetectBERT import DetectBERT
import os

model_path = f'{os.getcwd()}/data/DetectBERT/models/mpnet_cvefixes_w_masked'
embs_model_arch = 'sentence-transformers/all-MiniLM-L12-v2'
model = DetectBERT(embs_model_arch, num_labels=8) 

cuda:1


In [3]:
model.from_pretrained(model_path) 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


/home/jupyter-iec_chau/.local/lib/python3.10/site-packages/transformers/modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_fil

In [4]:
source = """
import os

def get_user_info(username):
    conn = sqlite3.connect('users.db')
    cursor = conn.cursor()

    # Vulnerable to SQL injection
    query = f"SELECT * FROM users WHERE username = '{username}'"
    cursor.execute(query)
    
    user_info = cursor.fetchone()
    conn.close()
    
    return user_info

"""

In [5]:
# source_path = f'/home/jupyter-iec_chau/DetectBERT/examples/remote_code_execution-.py'
# source_path = f'/home/jupyter-iec_chau/DetectBERT/examples/xsrf-2.py'
# source = read_file(source_path)
# print_with_lines(source)
pipeline = DetectBERTPythonPipeline(model, source)
out = pipeline.predict()
# print(pipeline.funcs[['content','lines']])

In [6]:
# out = pipeline.predict()
out

{}

In [7]:
from datasets import load_dataset
from DetectBERT import DetectBERTTrainerConfig, DetectBERTTrainer
ds = load_dataset('EddieChen372/CVEFixes_Python_with_norm_vul_lines')
model_ckt = 'sentence-transformers/all-mpnet-base-v2'
ds = ds.map(cast_type)

config = DetectBERTTrainerConfig(
      ds=ds,
      model_ckt=model_ckt,
      num_layers=3,
      epoch_num=100,
      learning_rate=1e-5,
      device=torch.device('cuda:1'),
      masked=True,
      save_each=25,
      architecture='mpnet',
      heads=12,
      func_cls=False
) 
trainer = DetectBERTTrainer(config=config, log=True)
trainer.load(f'{os.getcwd()}/data/DetectBERT/models/mpnet_cvefixes_w_masked')
   
trainer.evaluate()

tensor([ 0.1694, 75.8088, 50.8717, 85.9167, 32.2188, 50.5392])
cuda:1


/home/jupyter-iec_chau/.local/lib/python3.10/site-packages/transformers/modeling_utils.py:484: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_fil

embs num params: 109870464
cls num params: 43322118


  0%|          | 0/1146 [00:00<?, ?it/s]/home/jupyter-iec_chau/miniconda3/envs/venv/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
100%|██████████| 1146/1146 [00:03<00:00, 307.17it/s]


{'f1': 0.7520192861557007,
 'prec': 0.7850244641304016,
 'recall': 0.7263299226760864,
 'acc': 0.7263299226760864}

In [28]:
samples = []
for i, label in enumerate(ds['test']['label']):
   if not all(x == 0 for x in label) and 7 in label:
      source = ''.join(ds['test'][i]['raw_lines'])
      output = trainer.predict(ds['test'][i])
      if len(output) != 0:
         samples.append(output)

In [29]:
samples

[]

In [30]:
index = 0 
attentions = samples[index]['attentions']
lines = samples[index]['content']
labels = samples[index]['truth']

print(''.join(lines))
print(labels)

IndexError: list index out of range

In [ ]:
# @VAR_0.tag...
# """docstring"""
# VAR_5 = get_gravatar_url(VAR_1['request'], VAR_2, VAR_3)
# if VAR_5:
# return '<img src="%s" width="%s" height="%s" alt="%s"      class="gravatar"/>' % (
#     VAR_5, VAR_3, VAR_3, VAR_2.get_full_name() or VAR_2.username)
# return ''

In [26]:
from bertviz import head_view, model_view
model_view(attention=attentions, tokens=lines, display_mode='light')


<IPython.core.display.Javascript object>

In [13]:
head_view(attention=attentions, tokens=lines)

<IPython.core.display.Javascript object>

In [14]:
from transformers import AutoModel, AutoTokenizer


sample = """return safestring.mark_safe(rules)""" 
model = AutoModel.from_pretrained(model_path+ '/embs', output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained(model_path+ '/embs')
inputs = tokenizer.encode(sample, return_tensors='pt')
outputs = model(inputs)
embs_attentions = outputs.attentions

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
